<a href="https://colab.research.google.com/github/akhiltabjula/ComputationalMethodsForInformationSystems/blob/master/In_class_exercise_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 10/29/2020)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [169]:
# Write your code here
# from polyglot.text import Text

import pandas as pd
sentiments = pd.read_csv("/content/Sentiment Analysis and Text Classification.csv",encoding="ISO-8859-1")
sentiments=sentiments.drop("doc_id",axis=1)
sentiments["review"] = sentiments["review"].fillna(" ")
sentiments["sentiment"] = sentiments["sentiment"].apply(lambda x: str(x).lower()).fillna("Neutral")
sentiments.head(2)


,Names,review,cleaned_data,sentiment
0,MihaVrhunc,"Every once in a while a movie comes, that trul...",every movie comes truly makes impact joaquins ...,positive
1,lesterarnoldpinto,This is a movie that only those who have felt ...,movie felt alone isolated truly relate underst...,neutral


In [112]:
import nltk


In [113]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [114]:
#remove stop words 

from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [115]:
sentiments["cleaned_data"] = sentiments["cleaned_data"].apply(lambda x : " ".join([i for i in str(x).split(" ") if i not in list(stopwords)]))

In [116]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [117]:
#split the words and find the occurance of each word in the entire list
sentiments["set of words"] = sentiments["cleaned_data"].apply(lambda x : nltk.word_tokenize(str(x)))
sentiments.head(5)

,Names,review,cleaned_data,sentiment,set of words
0,MihaVrhunc,"Every once in a while a movie comes, that trul...",every movie comes truly makes impact joaquins ...,positive,"[every, movie, comes, truly, makes, impact, jo..."
1,lesterarnoldpinto,This is a movie that only those who have felt ...,movie felt alone isolated truly relate underst...,Neutral,"[movie, felt, alone, isolated, truly, relate, ..."
2,Aman_Goyal,"Truly a masterpiece, The Best Hollywood film o...",truly masterpiece best hollywood film 2019 one...,Positive,"[truly, masterpiece, best, hollywood, film, 20..."
3,logical_guy,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives tour de force performanc...,Positive,"[joaquin, phoenix, gives, tour, de, force, per..."
4,kdagoulis26,Most of the time movies are anticipated like t...,time movies anticipated like end falling short...,Neutral,"[time, movies, anticipated, like, end, falling..."


In [118]:

sentiment_list = list(sentiments[["cleaned_data","sentiment"]].to_records(index=False))
sentiment_list[:2]

[('every movie comes truly makes impact joaquins performance scenography brilliance grotesque haunting cringy hard watch times mesmerizing wont blink eye watching tragic seriously funny moments emotional rollercoaster sometimes multiple emotions poppingup timethis far typical actionriddled predictable superhero movie proper psychological thrillerdrama single best character development ever seen', 'positive'),
 ('movie felt alone isolated truly relate understand motive feel sorry character lot people see movie think encourages violence truly movie encourage every one us become better person treat everyone respect make feel like belong world instead making feel isolated', 'Neutral')]

In [119]:
sentences = sentiments["cleaned_data"].values.tolist()
list_of_words = [j for i in sentences for j in i.split(" ")]

In [120]:
def document_features(document):
    document_words = document.split(" ")
    features = {}
    for word in list_of_words:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [121]:
#accuracy
featuresets = [(document_features(str(d)), c) for (d,c) in sentiment_list]
train_set, test_set = featuresets[:50], featuresets[50:]
classifier = nltk.NaiveBayesClassifier.train(train_set)


In [122]:
print(nltk.classify.accuracy(classifier, test_set))

0.22


In [123]:
classifier.show_most_informative_features(20)

Most Informative Features
          contains(high) = True           neutra : positi =     15.0 : 1.0
       contains(started) = True           negati : positi =     15.0 : 1.0
         contains(loses) = True           negati : positi =     15.0 : 1.0
      contains(applause) = True           negati : positi =     15.0 : 1.0
       contains(looking) = True           negati : positi =     15.0 : 1.0
       contains(moments) = True           negati : positi =     15.0 : 1.0
         contains(genre) = True           negati : positi =     15.0 : 1.0
          contains(slow) = True           negati : positi =     15.0 : 1.0
 contains(psychological) = True           negati : positi =     15.0 : 1.0
      contains(terrible) = True           negati : positi =     15.0 : 1.0
      contains(complete) = True           negati : positi =     15.0 : 1.0
         contains(snaps) = True           negati : positi =     15.0 : 1.0
          contains(todd) = True           Negati : positi =     15.0 : 1.0

In [124]:
!pip install TextBlob

In [125]:
from textblob  import TextBlob
output_df = sentiments['set of words'].apply(lambda x : pd.value_counts(x))
output_df = output_df.fillna(0)
output_df1 = pd.DataFrame(output_df.columns)
output_df1["frequency"] = output_df.T.sum(axis=0)

In [126]:
output_df1 = output_df1[[0,"frequency"]].sort_values(by= "frequency", ascending=False).reset_index()
output_df1 = output_df1.drop("index",axis=1)

In [127]:
output_df1["rank"] = list(range(1,1699))

In [128]:
output_df1.head(10)

,0,frequency,rank
0,rule,378.0,1
1,single,371.0,2
2,scenography,200.0,3
3,psychological,193.0,4
4,person,154.0,5
5,makes,141.0,6
6,cringy,117.0,7
7,proper,116.0,8
8,multiple,115.0,9
9,making,104.0,10


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [157]:
# Write your code here
result2_df = sentiments[["cleaned_data","sentiment"]].reset_index()

result2_df["text blob polarity"] = result2_df["cleaned_data"].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
result2_df = result2_df.drop("index",axis=1)
result2_df.head(5)

,cleaned_data,sentiment,text blob polarity
0,every movie comes truly makes impact joaquins ...,positive,-0.027126
1,movie felt alone isolated truly relate underst...,Neutral,0.000000
2,truly masterpiece best hollywood film 2019 one...,Positive,0.236250
3,joaquin phoenix gives tour de force performanc...,Positive,-0.081481
4,time movies anticipated like end falling short...,Neutral,0.057143


In [158]:
result2_df['text_blob_prediction'] = pd.cut(result2_df['text blob polarity'], bins=5, labels=["negative","negative", "neutral", "positive","positive"],ordered=False)
result2_df.head(5)

,cleaned_data,sentiment,text blob polarity,text_blob_prediction
0,every movie comes truly makes impact joaquins ...,positive,-0.027126,negative
1,movie felt alone isolated truly relate underst...,Neutral,0.000000,negative
2,truly masterpiece best hollywood film 2019 one...,Positive,0.236250,neutral
3,joaquin phoenix gives tour de force performanc...,Positive,-0.081481,negative
4,time movies anticipated like end falling short...,Neutral,0.057143,negative


In [159]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [160]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [161]:

result2_df["vander positive polarity"] = result2_df["cleaned_data"].apply(lambda x: sia.polarity_scores(str(x))["pos"])
result2_df["vander neagtive polarity"] = result2_df["cleaned_data"].apply(lambda x: sia.polarity_scores(str(x))["neg"])
result2_df["vander neutral polarity"] = result2_df["cleaned_data"].apply(lambda x: sia.polarity_scores(str(x))["neu"])
result2_df["vander total polarity"] = result2_df["cleaned_data"].apply(lambda x: sia.polarity_scores(str(x))["compound"])

In [164]:
result2_df['vander_prediction'] = pd.cut(result2_df['vander total polarity'], bins=5, labels=["negative","negative", "neutral", "positive","positive"],ordered=False)
result2_df.head(5)

,cleaned_data,sentiment,text blob polarity,text_blob_prediction,vander positive polarity,vander neagtive polarity,vander neutral polarity,vander total polarity,vander_prediction
0,every movie comes truly makes impact joaquins ...,positive,-0.027126,negative,0.251,0.133,0.616,0.8519,positive
1,movie felt alone isolated truly relate underst...,Neutral,0.000000,negative,0.379,0.196,0.425,0.9042,positive
2,truly masterpiece best hollywood film 2019 one...,Positive,0.236250,neutral,0.339,0.115,0.545,0.9808,positive
3,joaquin phoenix gives tour de force performanc...,Positive,-0.081481,negative,0.198,0.086,0.717,0.6597,positive
4,time movies anticipated like end falling short...,Neutral,0.057143,negative,0.283,0.319,0.398,-0.5719,negative
